In [1]:
import ee
import geemap
import os
import geopandas as gpd

import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors

import jenkspy

import ipywidgets as widgets

ee.Initialize()

In [2]:
Map = geemap.Map(basemap="SATELLITE")

In [3]:
# PARAMETROS
RUTA_CATASTRO = 'projects/ee-bismarksr17/assets/CATASTRO240222'
CODIGO_PROPIEDAD = 506
FECHA_INICIO = '2021-05-01'
FECHA_FIN = '2021-05-15'

In [4]:
# cargar catastro desde engine
catastro = ee.FeatureCollection(RUTA_CATASTRO)
# filtra catastro por nombre de propiedad
prop = catastro.filter(ee.Filter.eq('COD_PROP', CODIGO_PROPIEDAD))\
                .filter(ee.Filter.neq('Variedad', 'RENOVACION'))
prop_geo = prop.geometry()
# get nombre de propiedad
nom_prop = prop.first().get('PROPIEDAD').getInfo()

# cargar coleccion de imagenes
coleccion = ee.ImageCollection('COPERNICUS/S2_SR')\
                .filterBounds(prop_geo)\
                .filterDate(FECHA_INICIO, FECHA_FIN)\
                .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 20)

# extraer IDs de la coleccion
ids = coleccion.reduceColumns(ee.Reducer.toList(), ['system:index']).get('list').getInfo()

# funcion para extraer fechas
def get_fechas(coll):
    def extraer_fecha(img):
        return ee.Image(img).date().format('dd/MM/YYYY')
    return coll.toList(coll.size(),0).map(extraer_fecha)

# extraer fechas
fechas = get_fechas(coleccion).getInfo()

# tabla de datos
dic = {'FECHA':fechas, 'ID':ids}
df = pd.DataFrame(dic)

# parametros de visualizacion
viz_img = {
    'bands':['RED','GREEN','BLUE'],
    'min':0.0,
    'max':0.5,
    'fillColorOpacity': '0'
}

vis_params = {
    'color': 'red', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

# cargar imagenes en geemap
for index, row in df.iterrows():
    img_aux = ee.Image('COPERNICUS/S2_SR/'+row[1])\
                .multiply(0.0001)\
                .select(['B2','B3','B4'], ['BLUE','GREEN','RED'])
    Map.addLayer(img_aux, viz_img, str(index) + ' - ' + row[0])

Map.centerObject(prop.geometry(), 15)
# Map.addLayer(prop, {'color':'red'}, nom_prop)
Map.addLayer(prop.style(**vis_params), {}, nom_prop)
print('CANTIDAD DE LOTES:', prop.size().getInfo())
print('CANTIDAD DE IMAGENES:', coleccion.size().getInfo())
print(df)
Map

CANTIDAD DE LOTES: 22
CANTIDAD DE IMAGENES: 4
        FECHA                                      ID
0  02/05/2021  20210502T141731_20210502T141831_T20KMF
1  02/05/2021  20210502T141731_20210502T141831_T20KMG
2  02/05/2021  20210502T141731_20210502T141831_T20KNF
3  02/05/2021  20210502T141731_20210502T141831_T20KNG


Map(center=[-17.231264716493012, -62.982816252838795], controls=(WidgetControl(options=['position', 'transpare…

In [12]:
x = DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 24pt">Bismark</div>',
        )

In [14]:
Map.addLayer(x)

AttributeError: 

The image argument in 'addLayer' function must be an instance of one of ee.Image, ee.Geometry, ee.Feature or ee.FeatureCollection.

In [8]:
import folium
from folium.features import DivIcon

m = folium.Map([34.0302, -118.2352], zoom_start=13)
folium.map.Marker(
    [34.0302, -118.2352],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 24pt">Bismark</div>',
        )
    ).add_to(m)
m

In [5]:
# index de la imagen a analizar
COD_IMAGEN = 0

In [6]:
# extrae el ID de la imagen de interes
id_imagen = df.loc[COD_IMAGEN]['ID']

# carga la imagen con las bandas de interes
img = ee.Image('COPERNICUS/S2_SR/'+id_imagen)\
        .multiply(0.0001)\
        .select(['B2','B3','B4','B8'], ['BLUE','GREEN','RED','NIR'])

In [7]:
# crea el NDVI y corta el NDVI
NDVI = img.normalizedDifference(['NIR','RED']).rename("NDVI")
NDVI_clip = NDVI.clip(prop_geo)

In [8]:
# crea un sample de los valore de pixel del NDVI
NDVI_values = NDVI.sampleRegions(prop_geo)
# reduce el resultado a valores de NDVI
pixel_values = NDVI_values.reduceColumns(ee.Reducer.toList(),['NDVI']).get('list').getInfo()
# aplica metodo Jenks
breaks = jenkspy.jenks_breaks(pixel_values, nb_class=8)
#print('Clases=', len(breaks))
#breaks

Clases= 9


In [9]:
# clasificacion del NDVI con los valors de Jenks
NDVI_class = ee.Image(-1).where(NDVI.lt(breaks[1]),1)\
                        .where(NDVI.gte(breaks[1]),2)\
                        .where(NDVI.gte(breaks[2]),3)\
                        .where(NDVI.gte(breaks[3]),4)\
                        .where(NDVI.gte(breaks[4]),5)\
                        .where(NDVI.gte(breaks[5]),6)\
                        .where(NDVI.gte(breaks[6]),7)\
                        .where(NDVI.gte(breaks[7]),8)

# recorta la clasificacion
NDVI_class_clip = NDVI_class.clip(prop_geo)

In [10]:
clasify = NDVI_class_clip.reproject(crs="EPSG:32720", scale=10)

In [11]:
# parametros de visualizacion
viz_class = {
    'min':8,
    'max':1,
    'palette': ['006100','498a00','8bb500','d6e600','ffe500','ffa600','ff6f00','ff2200']
}
viz_ndvi = {
    'palette': ['ff2200','ff6f00','ffa600','ffe500','d6e600','8bb500','498a00','006100'],
    'min': 0.1,
    'max': 0.7,
    'bands': 'NDVI'
}

In [12]:
Map.addLayer(NDVI_clip, viz_ndvi, 'NDVI')
Map.addLayer(clasify, viz_class, 'Class')
#Map

In [13]:
#clasify.projection().getInfo()

In [14]:
vector = clasify.reduceToVectors(**{
    'geometry': prop_geo,
    'crs': clasify.projection(),
    'scale': 10,
    'geometryType': 'polygon',
    'eightConnected': False
})

In [15]:
#Map.addLayer(vector, {'color':'green'}, 'RasterToVector')
#Map

In [16]:
vectorList = prop.toList(prop.size())

lista = ee.List([])

def function_1(feature, lista):    
    feature = ee.Feature(feature)
    lista = ee.List(lista)
    
    def function_2(feat):
        feat = ee.Feature(feat)
        intersection = feat.intersection(feature, ee.ErrorMargin(1))
        return ee.Feature(intersection).set({'area_2':intersection.area().divide(10000), 'class':feature.get('label')})
    
    intersection = vectorList.map(function_2)
    return lista.add(intersection)

propIntersect = vector.iterate(function_1, lista)

p = ee.FeatureCollection(ee.List(propIntersect).flatten())

In [17]:
#Map.addLayer(p, {'color':'green'}, 'Intersect')

In [18]:
def filtro(feature):
    return feature.set('geometryType', feature.geometry().type())

In [19]:
add_geometry = p.map(filtro)

In [20]:
res = add_geometry.filterMetadata('geometryType','contains','Polygon')

In [21]:
#Map.addLayer(res, {'red':'blue'},'Intersect Filter')

In [22]:
datos = res.toList(res.size()).getInfo()

lista_properties = []
for dato in datos:
    lista_properties.append(dato['properties'])
    
tabla = pd.DataFrame(lista_properties)
#tabla.head()

In [23]:
area_01 = tabla['area_2'].sum()
area_02 = prop.reduceColumns(ee.Reducer.sum(),['SUPERFICIE']).getInfo()['sum']

area_diff = (area_02 - area_01)/len(tabla)

tabla['area_2'] = tabla['area_2'] + area_diff
#tabla['area_2'].sum()

In [24]:
trunc = lambda x: math.trunc(100 * x) / 100;

In [25]:
estimativa = [0,20,30,40,50,60,70,80]
categoria = ['Nulo','Muy Bajo','Bajo Medio','Bajo','Medio','Medio Alto','Alto','Muy Alto']
colores = ['','','','','','','','']

def styling_specific_cell(x,row_idx,col_idx):
    color = ['background-color: #ff2200;',
             'background-color: #ff6f00;',
             'background-color: #ffa600;',
             'background-color: #ffe500;',
             'background-color: #d6e600;',
             'background-color: #8bb500;',
             'background-color: #498a00;',
             'background-color: #006100;']
    df_styler = pd.DataFrame('', index=x.index, columns=x.columns)
    df_styler.iloc[row_idx, col_idx] =  color
    return df_styler

def set_blod_categori(x,row_idx,col_idx):
    color = ['font-weight:bold;',
             'font-weight:bold;',
             'font-weight:bold;',
             'font-weight:bold;',
             'font-weight:bold;',
             'font-weight:bold;',
             'font-weight:bold;',
             'font-weight:bold;',
             'font-weight:bold;']
    df_styler = pd.DataFrame('', index=x.index, columns=x.columns)
    df_styler.iloc[row_idx, col_idx] =  color
    return df_styler

def set_blod_row(x,row_idx,col_idx):
    color = 'font-weight:bold; background-color: #9BC2E6; color: black;'
    df_styler = pd.DataFrame('', index=x.index, columns=x.columns)
    df_styler.iloc[row_idx, col_idx] =  color
    return df_styler

headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #9BC2E6; color: black; border:solid 1px;'
}

In [26]:
dina = pd.pivot_table(tabla, values='area_2', index=['class'], aggfunc=np.sum)
dina.rename(columns={'area_2':'SUPERFICIE'}, inplace=True)
dina.insert(loc=0, column='CATEGORIA', value=categoria)
dina.insert(loc=1, column='ESTIMATIVA', value=estimativa)
dina.insert(loc=1, column='COLOR', value=colores)

dina['%']=(dina['SUPERFICIE']/dina['SUPERFICIE'].sum())*100
dina['PRODUCCIÓN'] = dina['ESTIMATIVA']*dina['SUPERFICIE']

#dina = dina.rename(index={1:'Nulo', 2:'Muy Bajo', 3:'Bajo Medio', 4:'Bajo', 5:'Medio', 6:'Medio Alto', 7:'Alto', 8:'Muy Alto'})

dina.loc['Total'] = dina.sum(axis=0)
dina.at['Total','ESTIMATIVA'] = dina.at['Total','PRODUCCIÓN']/dina.at['Total','SUPERFICIE']
dina.at['Total','CATEGORIA'] = 'TOTAL'
dina.at['Total','COLOR'] = ''
#dina['CATEGORIA'] = dina['CATEGORIA'].fillna(0)
#dina = dina.applymap(trunc)

formato = {'ESTIMATIVA':'{:.2f}', 'SUPERFICIE':'{:.2f}', '%':'{:.2f} %', 'PRODUCCIÓN':'{:.2f}'}
i = pd.IndexSlice[dina.loc[(dina['%']<99.999999)].index, '%']
s = dina.style.format(formato)\
                .bar(subset=i, color='#10CB23')\
                .hide_index()\
                .apply(styling_specific_cell, row_idx = [0,1,2,3,4,5,6,7], col_idx = 1, axis = None)\
                .apply(set_blod_categori, row_idx = [0,1,2,3,4,5,6,7,8], col_idx = 0, axis = None)\
                .apply(set_blod_row, row_idx = 8, col_idx = [0,1,2,3,4,5], axis = None)\
                .set_table_styles([headers])
s = s.set_properties(border="1px solid black")

s

CATEGORIA,COLOR,ESTIMATIVA,SUPERFICIE,%,PRODUCCIÓN
Nulo,,0.00,27.07,11.16 %,0.00
Muy Bajo,,20.00,7.92,3.27 %,158.50
Bajo Medio,,30.00,3.00,1.24 %,90.10
Bajo,,40.00,8.29,3.42 %,331.63
Medio,,50.00,31.44,12.96 %,1572.12
Medio Alto,,60.00,55.71,22.95 %,3342.51
Alto,,70.00,60.79,25.05 %,4255.63
Muy Alto,,80.00,48.46,19.97 %,3877.12
TOTAL,,56.00,242.70,100.00 %,13627.60


In [122]:
# tabla dinamica
dina2 = pd.pivot_table(tabla, values='area_2', columns=['class'], index=['LOTES'], aggfunc=np.sum)
# cambiar calores NaN
dina2 = dina2.fillna(0)

# digitos validos para ordenar lotes
digitos_validos = ['0','1','2','3','4','5','6','7','8','9','.']
def eliminar_letras(lote):
    numero = ''
    for letra in lote:
        if letra in digitos_validos:
            numero = numero + letra
    return float(numero)

# copiar el indice en una nueva columna 'orden'
dina2['orden'] = dina2.index
# extraer valores numericos de los lotes en una nueva columna
dina2['numeracion'] = dina2['orden'].map(eliminar_letras)
# ordenar por numeracion de lote
dina2 = dina2.sort_values(by=['numeracion'])
# eliminar las columnas auxiliares
dina2 = dina2.drop(['orden','numeracion'], axis=1)

# suma total por filas
dina2['TOTAL'] = dina2.sum(axis=1)
# suma total por columnas
dina2.loc['TOTAL'] = dina2.sum(axis=0)

# convercion a procentajes totales por filas
dina2[1] = dina2[1]/dina2['TOTAL']*100
dina2[2] = dina2[2]/dina2['TOTAL']*100
dina2[3] = dina2[3]/dina2['TOTAL']*100
dina2[4] = dina2[4]/dina2['TOTAL']*100
dina2[5] = dina2[5]/dina2['TOTAL']*100
dina2[6] = dina2[6]/dina2['TOTAL']*100
dina2[7] = dina2[7]/dina2['TOTAL']*100
dina2[8] = dina2[8]/dina2['TOTAL']*100
dina2['TOTAL'] = dina2['TOTAL']/dina2['TOTAL']*100

# renombrar columnas
dina2.rename(columns={1:'Nulo', 2: 'Muy Bajo', 3:'Bajo Medio', 4:'Bajo', 5:'Medio', 6:'Medio Alto', 7:'Alto', 8:'Muy Alto'}, inplace=True)
# eliminar columna Total
dina2 = dina2.drop(['TOTAL'], axis=1)
# formato de los valores
formato_2 = {'Nulo':'{:.2f}%', 'Muy Bajo':'{:.2f}%', 'Bajo Medio':'{:.2f}%', 'Bajo':'{:.2f}%', 'Medio':'{:.2f}%', 'Medio Alto':'{:.2f}%', 'Alto':'{:.2f}%', 'Muy Alto':'{:.2f}%'}

dina2 = dina2.rename_axis('LOTES', axis=1)
dina2 = dina2.rename_axis(None, axis=0)

# CONSTRUCCION DE CMAP PERSONALISADO
# funcion para convertir valores RGB a valores 0 a 1
# https://towardsdatascience.com/simple-steps-to-create-custom-colormaps-in-python-f21482778aa2
def inter_from_256(x):
    return np.interp(x=x,xp=[0,255],fp=[0,1])
# construccion de diccionario para rango de colores
cdict = {
    'red':((0.0,inter_from_256(248),inter_from_256(248)),
           (1/5*1,inter_from_256(233),inter_from_256(233)),
           (1/5*2,inter_from_256(200),inter_from_256(200)),
           (1/5*3,inter_from_256(167),inter_from_256(167)),
           (1/5*4,inter_from_256(133),inter_from_256(133)),
           (1.0,inter_from_256(99),inter_from_256(99))),
    'green': ((0.0, inter_from_256(105), inter_from_256(105)),
            (1/5*1, inter_from_256(229), inter_from_256(229)),
            (1/5*2, inter_from_256(221), inter_from_256(221)),
            (1/5*3, inter_from_256(210), inter_from_256(210)),
            (1/5*4, inter_from_256(199), inter_from_256(199)),
            (1.0, inter_from_256(190), inter_from_256(190))),
    'blue': ((0.0, inter_from_256(107), inter_from_256(107)),
              (1/5*1, inter_from_256(130), inter_from_256(130)),
              (1/5*2, inter_from_256(130), inter_from_256(130)),
              (1/5*3, inter_from_256(128), inter_from_256(128)),
              (1/5*4, inter_from_256(125), inter_from_256(125)),
              (1.0, inter_from_256(123), inter_from_256(123))),
}

# contrucion de nuevo CMAP
new_cmap = colors.LinearSegmentedColormap('new_cmap',segmentdata=cdict)

# aplicacion de formato a toda la tabla 
tabla_formato = dina2.style.background_gradient(new_cmap, axis=None, text_color_threshold=0)\
                            .format(formato_2)\
                            .set_table_styles([headers])


tabla_formato = tabla_formato.set_properties(border="1px solid black")
tabla_formato

LOTES,Nulo,Muy Bajo,Bajo Medio,Bajo,Medio,Medio Alto,Alto,Muy Alto
L1,0.09%,0.73%,2.41%,2.64%,5.59%,20.32%,34.69%,33.52%
L2,0.00%,1.27%,1.48%,2.54%,5.67%,14.20%,43.14%,31.70%
L3,0.13%,1.25%,2.23%,2.34%,4.32%,3.87%,25.65%,60.20%
L4,0.00%,0.00%,0.00%,0.77%,6.98%,15.75%,32.63%,43.88%
L5,0.00%,0.28%,1.48%,1.42%,3.43%,24.55%,64.07%,4.77%
L6,0.00%,0.00%,0.03%,0.45%,2.18%,6.24%,29.77%,61.33%
L9,79.39%,20.50%,0.11%,0.00%,0.00%,0.00%,0.00%,0.00%
L10,0.00%,0.00%,18.88%,58.45%,21.30%,1.37%,0.00%,0.00%
L11,0.00%,0.00%,0.21%,29.56%,30.83%,24.88%,13.92%,0.60%
L12,0.01%,0.20%,1.66%,17.01%,36.06%,33.88%,11.18%,0.00%
